In [1]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import nba_py, sqlite3, os, time, random
from nba_py import Scoreboard, player, game, team

NBAchampions = {'1998-99':'SAS', '1999-00':'LAL', '2000-01':'LAL', '2001-02':'LAL', '2002-03':'SAS','2003-04':'DET', '2004-05':'SAS','2005-06':'MIA', '2006-07':'SAS','2007-08':'BOS', '2008-09':'LAL', '2009-10':'LAL', '2010-11':'DAL','2011-12':'MIA', '2012-13':'MIA','2013-14':'SAS', '2014-15':'GSW','2015-16':'CLE', '2016-17':'GSW'}

def targetSeason(targetTime):
    if 10 <= targetTime.month <= 12:
        targetSeason0 = str(targetTime.year) + '-' + str(targetTime.year + 1)[2:4]
    elif 1 <= targetTime.month <= 6:
        targetSeason0 = str(targetTime.year - 1) + '-' + str(targetTime.year)[2:4]
    else:
        return 'time not in game season or time format error'
    return targetSeason0

def getDefendingChampion(targetTime): #取上赛季,为求卫冕冠军名
    if 10 <= targetTime.month <= 12:
        targetSeason0 = str(targetTime.year - 1) + '-' + str(targetTime.year)[2:4]
    elif 1 <= targetTime.month <= 6:
        targetSeason0 = str(targetTime.year - 2) + '-' + str(targetTime.year - 1)[2:4]
    else:
        return 'time not in game season or time format error'
    return NBAchampions[targetSeason0]

targetTeam = 'ORL'
targetTime = datetime(2016,11,21)
targetSeason1 = targetSeason(targetTime)
pd.set_option('display.max_columns',60)

teamList = team.TeamList(league_id='00').info()
teamID = teamList[teamList['ABBREVIATION'] == targetTeam]['TEAM_ID']
teamGames = team.TeamGameLogs(teamID, season=targetSeason1, season_type='Regular Season').info()
teamGames[:3]
# teamGames['GAME_DATE'] = pd.to_datetime(teamGames['GAME_DATE'])
# gameID = teamGames[teamGames['GAME_DATE'] == pd.to_datetime(targetTime.date())]['Game_ID']

# teamPerformance = team.TeamPerformanceSplits(teamID,season='2017-18').score_differential()[['GROUP_VALUE','GROUP_VALUE_2','GP','W','L','PTS','PLUS_MINUS']]
# teamPerformance['OPPO_PTS'] = teamPerformance['PTS'] + teamPerformance['PLUS_MINUS']
# lastNgames = team.TeamLastNGamesSplits(teamID,season=targetSeason1).last5()[['GROUP_VALUE','W','L','PTS','PLUS_MINUS']]
# lastNgames['OPPO_PTS'] = lastNgames['PTS'] + lastNgames['PLUS_MINUS']

# game0 = Scoreboard(month=targetTime.month, day=targetTime.day, year=targetTime.year, league_id='00', offset=0)
# #game0.game_header() .line_score() .series_standings() .last_meeting() .east_conf_standings_by_day() .west_conf_standings_by_day() .available()
# game0.line_score()[['GAME_DATE_EST', 'GAME_SEQUENCE', 'GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION','PTS']]
# game1 = game.Boxscore(gameID, season=targetSeason1, season_type='Regular Season', range_type='0', start_period='0', end_period='0', start_range='0', end_range='0')
# #game1.player_stats() .team_stats() .team_starter_bench_stats()
# game1.team_stats()[['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION','PTS', 'PLUS_MINUS']]
# game2 = game.BoxscoreSummary(gameID, season=targetSeason1, season_type='Regular Season', range_type='0', start_period='0', end_period='0', start_range='0', end_range='0')
# #game2.game_summary() .other_stats() .officials() .inactive_players() .game_info() .line_score() .last_meeting() .season_series() .available_video()
# game2.line_score()[['GAME_DATE_EST', 'GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'PTS']]

# player0 = team.TeamCommonRoster(teamID, season=targetSeason1).roster()#coaches()
# player1 = team.TeamPlayers(teamID, measure_type='Base', per_mode='PerGame', plus_minus='N', pace_adjust='N', rank='N', league_id='00', season=targetSeason1, season_type='Regular Season', po_round='0', outcome='', location='', month='0', season_segment='', date_from='', date_to='', opponent_team_id='0', vs_conference='', vs_division='', game_segment='', period='0', shot_clock_range='', last_n_games='0').season_totals()
# player2 = game.PlayerTracking(gameID).info()'''

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612753,0021601217,"APR 12, 2017",ORL vs. DET,W,29,53,0.354,240,42,87,0.483,13,25,0.520,16,17,0.941,6,45,51,27,7,6,11,22,113
1,1610612753,0021601206,"APR 10, 2017",ORL @ CHI,L,28,53,0.346,240,29,84,0.345,5,27,0.185,12,16,0.750,9,28,37,18,5,5,16,18,75
2,1610612753,0021601190,"APR 08, 2017",ORL vs. IND,L,28,52,0.350,240,41,84,0.488,14,30,0.467,16,22,0.727,4,29,33,29,10,2,13,16,112


In [2]:
def getNBAscores(searchDate,searchTeam=None):
    game0 = Scoreboard(month=searchDate.month, day=searchDate.day, year=searchDate.year, league_id='00', offset=0)
    #game0.game_header() .line_score() .series_standings() .last_meeting() .east_conf_standings_by_day() .west_conf_standings_by_day() .available()
    if not (game0.line_score().empty or game0.line_score()['PTS'][0]==None):#有的日期没有比赛，却被None填充，非空表，须排除
        scorelist0 = game0.line_score()[['GAME_DATE_EST', 'GAME_SEQUENCE', 'GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION','PTS']]
        scorelist0 = scorelist0.sort_values(by='GAME_ID')#有些日期的GAME_ID排序混乱，坑啊
        scorelist = scorelist0.set_index(np.arange(scorelist0.shape[0]))#重新索引
        scorelist['GAME_DATE_EST'] = pd.to_datetime(scorelist['GAME_DATE_EST']).dt.date #此行及下行为获得字符串格式的日期
        scorelist['GAME_DATE_EST'] = scorelist['GAME_DATE_EST'].apply(lambda x:str(x))
        leftTable = scorelist[scorelist.index%2 == 0] #此及以下两行为合并奇偶数行
        rightTable = scorelist[scorelist.index%2 == 1]
        scorelist1 = pd.merge(leftTable,rightTable,how = 'inner',on=['GAME_DATE_EST','GAME_SEQUENCE','GAME_ID'])
        scorelist2 = scorelist1.iloc[:,[0,2,3,6,4,7,5,8]]
        def f1(x):
            return 3 if x[-1]>x[-2] else 0
        scorelist2['w_l'] = scorelist2.apply(f1, axis=1)
        scorelist2['margin'] = scorelist2.iloc[:,-2] - scorelist2.iloc[:,-3]
        if (searchTeam in list(scorelist2.iloc[:,-6])) or (searchTeam in list(scorelist2.iloc[:,-5])):
            return scorelist2[(scorelist2.iloc[:,-6]==searchTeam)|(scorelist2.iloc[:,-5]==searchTeam)].values
        elif searchTeam is None:
            return scorelist2.values
        else:
            return [[None],['%s has no game in %s'%(searchTeam,searchDate.date())]]#便于判断是否空表
    else:
        return [[None],[None]]#便于判断是否空表
    
# searchDate = datetime(2003,12,25)
# searchTeam = 'Non'#'LAC'
# getNBAscores(searchDate,searchTeam)

In [3]:
sqlPath = os.getcwd().replace('jupyter_notebook', 'web_spiders\\NBA_from_2010.db')
connection = sqlite3.connect(sqlPath)
cursor = connection.cursor()
table_name = 'NBA_scores_from_1998'

cursor.execute('CREATE TABLE IF NOT EXISTS %s ' % table_name + '''(
                game_date_EST TEXT,
                game_ID       TEXT,
                road_team_ID  NUMERIC,
                home_team_ID  NUMERIC,
                road_team     TEXT,
                home_team     TEXT,
                road_points   NUMERIC,
                home_points   NUMERIC,
                w_l           NUMERIC,
                margin        NUMERIC)''')

def saveToDB(fetchTable,table_name):
    #写入前查重，用game_ID保证唯一性
    cursor.execute('SELECT game_ID FROM %s' % table_name)
    sql0 = cursor.fetchall()
    if fetchTable[0][0]: #去除无比赛的数据
        if not (tuple(fetchTable[0][1]) in sql0):
            cursor.executemany('INSERT INTO %s ' % table_name + 'VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', fetchTable)
    connection.commit()

In [5]:
# #根据日期爬取比赛比分
# for NBAyear in range(2018,2019):
#     searchDate = datetime(NBAyear,1,25)
#     while searchDate <= datetime(NBAyear,1,30):
#         saveToDB(getNBAscores(searchDate),table_name)
#         print(searchDate)
#         searchDate += timedelta(days = 1)
# #去除完全重复的行（根据game_ID）
# cursor.execute('DELETE FROM %s WHERE rowid NOT IN (SELECT MAX(rowid) rowid FROM %s GROUP BY game_ID)'%(table_name,table_name))
# connection.commit()
# # cursor.execute('DELETE FROM NBA_scores_from_1998 WHERE game_date_EST == "2018-01-25"')
# # connection.commit()

2018-01-25 00:00:00
2018-01-26 00:00:00
2018-01-27 00:00:00
2018-01-28 00:00:00
2018-01-29 00:00:00
2018-01-30 00:00:00


In [ ]:
table_name = 'NBA_scores_from_1998' #1999年起卫冕冠军的圣诞大战战绩
cursor.execute('SELECT * FROM {} WHERE strftime("%m-%d",game_date_EST) == "12-25"'.format(table_name))#%s占位符与strftime的格式化字符冲突，改用.format方法
sql = cursor.fetchall()
col_name_list = [tuple[0] for tuple in cursor.description]
df = pd.DataFrame(sql, columns=col_name_list)
df['game_date_EST'] = pd.to_datetime(df['game_date_EST'])
df01 = df['game_date_EST'].apply(getDefendingChampion)
df02 = df[(df['road_team']==df01)|(df['home_team']==df01)]
df02

In [ ]:
table_name = 'NBA_odds_from_2010' #2010年起卫冕冠军的圣诞大战战绩及odds
cursor.execute('SELECT * FROM {} WHERE strftime("%m-%d",game_time) == "12-26"'.format(table_name))#%s占位符与strftime的格式化字符冲突，改用.format方法
sql1 = cursor.fetchall()
col_name_list1 = [tuple[0] for tuple in cursor.description]
df1 = pd.DataFrame(sql1, columns=col_name_list1)
df1['game_time'] = pd.to_datetime(df1['game_time'])
df10 = df1['game_time'].apply(getDefendingChampion)
df11 = df1[(df1['road_team']==df10)|(df1['home_team']==df10)]
df11 #df1

In [609]:
table_name = 'NBA_odds_from_2010' #2010年起一天内有至少两场比赛 margin_odds >= XX 的日期
cursor.execute('SELECT * FROM %s WHERE margin_odds >= 50' % table_name)
sql2 = cursor.fetchall()
col_name_list2 = [tuple[0] for tuple in cursor.description]

df2 = pd.DataFrame(sql2, columns=col_name_list2)
df2['game_time'] = pd.to_datetime(df2['game_time'])
#df2 = df2.reset_index(drop=True)
#df2.xs('2011', level ='game_time')
duplicatedDays = df2['game_time'].dt.date.duplicated(keep=False) #同一天内
df20 = df2[duplicatedDays]#.iloc[:,[0,1,2,3,4,8,9,12,13,15,16]].round(1)
df20

,game_time,road_team,home_team,road_points,home_points,w_l,money_line_final,spread_init,spread_final,spread_w_l,spread_odds_final,totals_init,totals_final,totals_w_l,totals_odds_final,margin,margin_odds
5,2012-01-30 07:00:00,TOR,BKN,94,73,0,2.68,-4.5,-5.5,0,1.75,184.5,184.5,0,1.69,客21-25,60
6,2012-01-30 07:00:00,IND,ORL,106,85,0,2.42,-4.5,-4.5,0,1.75,182.5,181.5,3,1.81,客21-25,60
21,2013-03-13 07:00:00,BOS,CHA,74,100,3,6.30,10.5,11.5,3,1.81,189.5,189.5,0,1.75,主26+,200
22,2013-03-13 08:00:00,SAS,MIN,83,107,3,4.65,10.5,9.5,3,1.91,195.5,195.5,0,1.82,主21-25,120
40,2015-01-08 09:00:00,UTA,CHI,97,77,0,5.07,-10.5,-10.5,0,1.82,193.5,192.5,0,1.82,客16-20,60
41,2015-01-08 11:00:00,OKC,SAC,83,104,3,4.02,6.5,7.5,3,1.90,211.5,212.5,0,1.72,主21-25,75
44,2015-10-29 08:00:00,DEN,HOU,105,85,0,5.30,-10.5,-10.5,0,1.75,212.5,211.5,0,1.75,客16-20,62
45,2015-10-29 08:00:00,CLE,MEM,106,76,0,2.50,-5.5,-5.5,0,1.68,188.5,188.5,0,1.80,客26+,70
46,2015-10-29 08:00:00,NYK,MIL,122,97,0,2.50,-6.5,-5.5,0,1.68,191.5,194.5,3,1.75,客21-25,50
77,2017-11-19 08:00:00,UTA,ORL,125,85,0,2.92,-6.5,-6.5,0,1.75,207.5,206.5,3,1.75,客26+,75


In [ ]:
def get_NBA_conf_standings(searchDate): #获取实时排名
    teamList = team.TeamList(league_id='00').info()
    teamAbbreviation = teamList[['TEAM_ID','ABBREVIATION']][:30]
    game0 = Scoreboard(month=searchDate.month, day=searchDate.day, year=searchDate.year, league_id='00', offset=0)
    df01 = pd.merge(game0.east_conf_standings_by_day(), teamAbbreviation, how = 'left',on=['TEAM_ID'])
    df01 = df01[['TEAM_ID','ABBREVIATION','W', 'L']]
    df1 = df01.copy(deep=True)
    df02 = pd.merge(game0.west_conf_standings_by_day(), teamAbbreviation, how = 'left',on=['TEAM_ID'])
    df02 = df02[['TEAM_ID','ABBREVIATION','W', 'L']]
    df2 = df02.copy(deep=True)
    df2.columns = ['teamID_w','westTeam','W_w','L-w']
    df2[['teamID_e','eastTeam','W_e','L-e']] = df1
    df3 = pd.merge(df02, df01, how = 'outer')
    df3['W_rate'] = df3['W']/(df3['W'] + df3['L'])
    df3 = df3.sort_values(by='W_rate',ascending=False).reset_index(drop=True)
    return df3 #df2

get_NBA_conf_standings(datetime.now())

In [10]:
table_name = 'NBA_odds_from_2010' #2010年起 w_l_odds >=< XX 的日期
targetGameTime = '2016-10-10'
cursor.execute('SELECT * FROM {} WHERE spread_init <= -10 AND strftime("%Y-%m-%d",game_time) >= "{}"'.format(table_name, targetGameTime)) # 
sql3 = cursor.fetchall()
col_name_list3 = [tuple[0] for tuple in cursor.description]

df3 = pd.DataFrame(sql3, columns=col_name_list3)
df3['game_time'] = pd.to_datetime(df3['game_time'])
df3['spreadWL'] = df3['road_points'] - df3['home_points'] - df3['spread_init']
df3['spreadWL'] = df3['spreadWL'].apply(lambda x:0 if x>0 else 3)
df3['totalsWL'] = df3['road_points'] + df3['home_points'] - df3['totals_init']
df3['totalsWL'] = df3['totalsWL'].apply(lambda x:3 if x>0 else 0)
df3 = df3.iloc[:,[0,1,2,3,4,7,17,8,9,11,18,12,13]].round(1)

df30 = df3[df3['spread_init'] < -22]['totalsWL'].value_counts()
for i in range(-21,-9):
    dfj = df3[df3['spread_init'] < i]['totalsWL'].value_counts()
    df30 = pd.concat([df30,dfj],axis=1)

df30.columns = range(-22,-9)
df30 = df30.fillna(0)
df30.loc['0_rate'] = df30.loc[0]/(df30.loc[0] + df30.loc[3])
pd.set_option('precision', 3)
df30

,-22,-21,-20,-19,-18,-17,-16,-15,-14,-13,-12,-11,-10
0,0.0,1.0,1.0,1.0,3.00,3.000,8.0,14.000,26.000,44.000,64.000,91.000,132.000
3,0.0,0.0,0.0,1.0,1.00,6.000,8.0,18.000,32.000,49.000,73.000,108.000,151.000
0_rate,NaN,1.0,1.0,0.5,0.75,0.333,0.5,0.438,0.448,0.473,0.467,0.457,0.466


In [613]:
table_name = 'NBA_odds_from_2010'
cursor.execute('SELECT * FROM %s WHERE spread_init >= 10' % table_name)
sql4 = cursor.fetchall()
col_name_list4 = [tuple[0] for tuple in cursor.description]

df4 = pd.DataFrame(sql4, columns=col_name_list4)
df4['game_time'] = pd.to_datetime(df4['game_time'])
df4['spreadWL'] = df4['road_points'] - df4['home_points'] - df4['spread_init']
df4['spreadWL'] = df4['spreadWL'].apply(lambda x:0 if x>0 else 3)
df4['totalsWL'] = df4['road_points'] + df4['home_points'] - df4['totals_init']
df4['totalsWL'] = df4['totalsWL'].apply(lambda x:3 if x>0 else 0)
df4 = df4.iloc[:,[0,1,2,3,4,7,17,8,9,11,18,12,13]].round(1)

df40 = df4[df4['spread_init'] > 10]['spreadWL'].value_counts()
for i in range(11,18):
    dfk = df4[df4['spread_init'] > i]['spreadWL'].value_counts()
    df40 = pd.concat([df40,dfk],axis=1)

df40.columns = range(10,18)
df40 = df40.fillna(0)
df40.loc['3_rate'] = df40.loc[3]/(df40.loc[0] + df40.loc[3])
pd.set_option('precision', 3)
df40

,10,11,12,13,14,15,16,17
0,69.000,36.000,17.000,8.000,4.000,0.0,0.0,0.0
3,75.000,55.000,28.000,13.000,7.000,4.0,3.0,0.0
3_rate,0.521,0.604,0.622,0.619,0.636,1.0,1.0,NaN


In [608]:
def expectedTotals(game_time,road_team,home_team,spread_handicap=0):
    
    gameSeason = targetSeason(game_time)
    dateTo = game_time.date()
    dateFrom = dateTo - timedelta(days=365)
    teamList = team.TeamList(league_id='00').info()
    
    roadID = teamList[teamList['ABBREVIATION'] == road_team]['TEAM_ID']
    homeID = teamList[teamList['ABBREVIATION'] == home_team]['TEAM_ID']
        
    roadPerformance = team.TeamPerformanceSplits(roadID,season=gameSeason,date_from=dateFrom,date_to=dateTo).score_differential()[['GROUP_VALUE','GROUP_VALUE_2','GP','W','L','PTS','PLUS_MINUS']]
    roadPerformance['OPPO_PTS'] = roadPerformance['PTS'] - roadPerformance['PLUS_MINUS']
    homePerformance = team.TeamPerformanceSplits(homeID,season=gameSeason,date_from=dateFrom,date_to=dateTo).score_differential()[['GROUP_VALUE','GROUP_VALUE_2','GP','W','L','PTS','PLUS_MINUS']]
    homePerformance['OPPO_PTS'] = homePerformance['PTS'] - homePerformance['PLUS_MINUS']
    
    nullPerformance  = pd.DataFrame([['W','All',0,0,0],['L','All',0,0,0]],columns=['GROUP_VALUE','GROUP_VALUE_2','GP','PTS','OPPO_PTS'])
    nullPerformanceW = pd.DataFrame([['W','All',0,0,0]],columns=['GROUP_VALUE','GROUP_VALUE_2','GP','PTS','OPPO_PTS'])
    nullPerformanceL = pd.DataFrame([['L','All',0,0,0]],columns=['GROUP_VALUE','GROUP_VALUE_2','GP','PTS','OPPO_PTS'])
    
    if roadPerformance.empty:
        roadPerformance = nullPerformance
    if homePerformance.empty:
        homePerformance = nullPerformance
    
    roadPerformanceW = roadPerformance[roadPerformance['GROUP_VALUE']=='W'] if any(roadPerformance['GROUP_VALUE']=='W') else nullPerformanceW
    roadPerformanceL = roadPerformance[roadPerformance['GROUP_VALUE']=='L'] if any(roadPerformance['GROUP_VALUE']=='L') else nullPerformanceL
    homePerformanceW = homePerformance[homePerformance['GROUP_VALUE']=='W'] if any(homePerformance['GROUP_VALUE']=='W') else nullPerformanceW
    homePerformanceL = homePerformance[homePerformance['GROUP_VALUE']=='L'] if any(homePerformance['GROUP_VALUE']=='L') else nullPerformanceL
    
    roadWinAll  = roadPerformanceW[roadPerformanceW['GROUP_VALUE_2']=='All']
    roadLoseAll = roadPerformanceL[roadPerformanceL['GROUP_VALUE_2']=='All']
    roadMeanPTS = (roadWinAll['PTS'].values[0]*roadWinAll['GP'].values[0] + roadLoseAll['PTS'].values[0]*roadLoseAll['GP'].values[0])/(roadWinAll['GP'].values[0] + roadLoseAll['GP'].values[0])
    roadMeanOPPOPTS = (roadWinAll['OPPO_PTS'].values[0]*roadWinAll['GP'].values[0] + roadLoseAll['OPPO_PTS'].values[0]*roadLoseAll['GP'].values[0])/(roadWinAll['GP'].values[0] + roadLoseAll['GP'].values[0])
    
    homeWinAll  = homePerformanceW[homePerformanceW['GROUP_VALUE_2']=='All']
    homeLoseAll = homePerformanceL[homePerformanceL['GROUP_VALUE_2']=='All']
    homeMeanPTS = (homeWinAll['PTS'].values[0]*homeWinAll['GP'].values[0] + homeLoseAll['PTS'].values[0]*homeLoseAll['GP'].values[0])/(homeWinAll['GP'].values[0] + homeLoseAll['GP'].values[0])
    homeMeanOPPOPTS = (homeWinAll['OPPO_PTS'].values[0]*homeWinAll['GP'].values[0] + homeLoseAll['OPPO_PTS'].values[0]*homeLoseAll['GP'].values[0])/(homeWinAll['GP'].values[0] + homeLoseAll['GP'].values[0]) 

    if spread_handicap > 0:
        expectedTotals0 = roadMeanPTS + homeMeanOPPOPTS
    elif spread_handicap <= 0:
        expectedTotals0 = roadMeanOPPOPTS + homeMeanPTS
        
    teamScoreSplitsName = ['All','5 Points and Under','6-10 Points','11-15 Points','16-20 Points','Over 20 Points']  
    roadScoreSplitsIndex = homeScoreSplitsIndex = int(abs(spread_handicap)//5) + 1
    
    if abs(spread_handicap)>= 25:#超过20分的都算一档
        roadScoreSplitsIndex = homeScoreSplitsIndex = 5
   
    if spread_handicap > 0:     
        if all(roadPerformanceW['GROUP_VALUE_2'] != teamScoreSplitsName[roadScoreSplitsIndex]):
            roadScoreSplitsIndex = 0
        if all(homePerformanceL['GROUP_VALUE_2'] != teamScoreSplitsName[homeScoreSplitsIndex]):
            homeScoreSplitsIndex = 0
    elif spread_handicap <= 0:
        if all(roadPerformanceL['GROUP_VALUE_2'] != teamScoreSplitsName[roadScoreSplitsIndex]):
            roadScoreSplitsIndex = 0
        if all(homePerformanceW['GROUP_VALUE_2'] != teamScoreSplitsName[homeScoreSplitsIndex]):
            homeScoreSplitsIndex = 0
    
    roadWin  = roadPerformanceW[roadPerformanceW['GROUP_VALUE_2']==teamScoreSplitsName[roadScoreSplitsIndex]]
    roadLose = roadPerformanceL[roadPerformanceL['GROUP_VALUE_2']==teamScoreSplitsName[roadScoreSplitsIndex]]
    homeWin  = homePerformanceW[homePerformanceW['GROUP_VALUE_2']==teamScoreSplitsName[homeScoreSplitsIndex]]
    homeLose = homePerformanceL[homePerformanceL['GROUP_VALUE_2']==teamScoreSplitsName[homeScoreSplitsIndex]]
    
    if spread_handicap > 0:
        expectedTotals1 = (roadWin['PTS'].values[0] + homeLose['OPPO_PTS'].values[0] + homeLose['PTS'].values[0] + roadWin['OPPO_PTS'].values[0])/2
    else:
        expectedTotals1 = (roadLose['PTS'].values[0] + homeWin['OPPO_PTS'].values[0] + homeWin['PTS'].values[0] + roadLose['OPPO_PTS'].values[0])/2

    return expectedTotals0,expectedTotals1

# expectedTotals0 = []
# expectedTotals1 = []
# df4 = df4.iloc[2091:]
# for i in range(df4.shape[0]):
#     dfi = df4.iloc[i]
#     expectedTotal = expectedTotals(dfi['game_time'],dfi['road_team'],dfi['home_team'],spread_handicap=dfi['spread_init'])
#     time.sleep(random.uniform(10,50)/10)
#     expectedTotals0.append(expectedTotal[0])
#     expectedTotals1.append(expectedTotal[1])
# df4['expectedTotals0'] = expectedTotals0
# df4['expectedTotals1'] = expectedTotals1
# df4['totals_diff'] = df4['totals_init'] - df4['expectedTotals1']
# df4.to_csv ('odds_vs_expectedTotals_test.csv', encoding='utf-8')
# df4
df5 = pd.read_csv('odds_vs_expectedTotals_test.csv',index_col=0,encoding='utf-8')
df5['spreadWL'] = df5['road_points'] - df5['home_points'] - df5['spread_init']
df5['spreadWL'] = df5['spreadWL'].apply(lambda x:0 if x>0 else 3)
df5['totalsWL'] = df5['road_points'] + df5['home_points'] - df5['totals_init']
df5['totalsWL'] = df5['totalsWL'].apply(lambda x:3 if x>0 else 0)
df6 = df5.iloc[:,[0,1,2,3,4,7,20,11,21,18,19,15,16]].round(1)
df6

,game_time,road_team,home_team,road_points,home_points,spread_init,spreadWL,totals_init,totalsWL,expectedTotals1,totals_diff,margin,margin_odds
5,2012-01-30 07:00:00,TOR,BKN,94,73,-4.5,0,184.5,0,187.5,-3.0,客21-25,60
6,2012-01-30 07:00:00,IND,ORL,106,85,-4.5,0,182.5,3,195.5,-13.0,客21-25,60
21,2013-03-13 07:00:00,BOS,CHA,74,100,10.5,3,189.5,0,191.4,-2.0,主26+,200
22,2013-03-13 08:00:00,SAS,MIN,83,107,10.5,3,195.5,0,197.4,-1.9,主21-25,120
40,2015-01-08 09:00:00,UTA,CHI,97,77,-10.5,0,193.5,0,191.3,2.2,客16-20,60
41,2015-01-08 11:00:00,OKC,SAC,83,104,6.5,3,211.5,0,207.4,4.1,主21-25,75
44,2015-10-29 08:00:00,DEN,HOU,105,85,-10.5,0,212.5,0,0.0,212.5,客16-20,62
45,2015-10-29 08:00:00,CLE,MEM,106,76,-5.5,0,188.5,0,203.5,-15.0,客26+,70
46,2015-10-29 08:00:00,NYK,MIL,122,97,-6.5,0,191.5,3,106.5,85.0,客21-25,50
77,2017-11-19 08:00:00,UTA,ORL,125,85,-6.5,0,207.5,3,210.0,-2.6,客26+,75


In [ ]:
df6[(df6['spread_init'] > 12)&(df6['totals_diff'] <-3)]['spreadWL'].value_counts()
df6[df6['spread_init'] > 15]['spreadWL'].value_counts()

df60 = df6[df6['spread_init'] > 10]['totalsWL'].value_counts()
for i in range(11,17):
    dfm = df6[df6['spread_init'] > i]['totalsWL'].value_counts()
    df60 = pd.concat([df60,dfm],axis=1)

df60.columns = range(10,17)
df60 = df60.fillna(0)
df60.loc['3_rate'] = df60.iloc[0]/(df60.iloc[0] + df60.iloc[1])
pd.set_option('precision', 3)
df60

In [597]:
table_name = 'NBA_odds_from_2010'
cursor.execute('SELECT * FROM %s WHERE w_l != spread_w_l' % table_name) # spread_init >= 10
sql7 = cursor.fetchall()
col_name_list7 = [tuple[0] for tuple in cursor.description]

df7 = pd.DataFrame(sql7, columns=col_name_list7)
df7['game_time'] = pd.to_datetime(df7['game_time'])
df7['spreadWL'] = df7['road_points'] - df7['home_points'] - df7['spread_init']
df7['spreadWL'] = df7['spreadWL'].apply(lambda x:0 if x>0 else 3)
df7['totalsWL'] = df7['road_points'] + df7['home_points'] - df7['totals_init']
df7['totalsWL'] = df7['totalsWL'].apply(lambda x:3 if x>0 else 0)
df7 = df7.iloc[:,[0,1,2,3,4,7,17,8,9,11,18,12,13]].round(1)
df70 = df7['totalsWL'].value_counts()#[df7['spread_init'] < -1]
# for i in range(11,18):
#     dfk = df7[df7['spread_init'] > i]['spreadWL'].value_counts()
#     df70 = pd.concat([df70,dfk],axis=1)

# df70.columns = range(10,18)
# df70 = df70.fillna(0)
# df70.loc['3_rate'] = df70.loc[3]/(df70.loc[0] + df70.loc[3])
# pd.set_option('precision', 3)
df70

3    932
0    829
Name: totalsWL, dtype: int64